<a href="https://colab.research.google.com/github/victorgau/python_investment/blob/master/10-1%20%E5%96%AE%E7%AD%96%E7%95%A5%E5%A4%9A%E8%82%A1%E7%A5%A8%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 單策略多股票分析

In [ ]:
# 將需要的模組跟檔案放入 colab 中
!wget https://github.com/victorgau/python_investment/raw/master/backtest.py -o /dev/null
!wget https://github.com/victorgau/python_investment/raw/master/stockdata -o /dev/null
!wget https://github.com/victorgau/python_investment/raw/master/twstockdata -o /dev/null

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
# 載入計算回測指標的函式
from backtest import indicators

In [ ]:
def breakout(df):
    # Donchian Channel
    df['10d_low'] = pd.Series.rolling(df['Adj Close'], window=10).min()
    df['20d_high'] = pd.Series.rolling(df['Adj Close'], window=20).max()

    has_position = False
    df['signals'] = 0
    for t in range(1, df['signals'].size):
        if df['Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()

In [ ]:
def goldcross(df):
    df['5d'] = pd.Series.rolling(df['Adj Close'], window=10).mean()
    df['20d'] = pd.Series.rolling(df['Adj Close'], window=20).mean()

    has_position = False
    df['signals'] = 0
    for t in range(1, df['signals'].size):
        if df['5d'][t] > df['20d'][t] and df['5d'][t-1] <= df['20d'][t-1] and df['20d'][t-1] <= df['20d'][t]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Adj Close'][t] < df['20d'][t] and df['Adj Close'][t-1] > df['20d'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()

In [ ]:
# 從哪一個檔案讀取資料
filename = 'twstockdata'

# 使用哪一個策略
strategy = goldcross

In [ ]:
# 讀出預先下載好的股價資料
with open(filename, 'rb') as f:
    data = pickle.load(file=f)

# 計算各支股票的回測結果
results = []

for symbol in data:
    try:
        strategy(data[symbol])
        if np.all(data[symbol]['signals']==0):
            print("Symbol:", symbol, "沒有出現買賣訊號。")
            continue
        SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
        days = (data[symbol].index[-1] - data[symbol].index[0]).days
        results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                        data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol))
    except Exception as e:
        print("Error occurs at symbol:", symbol, "==>", e.args)

results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','days', 'entries','symbol'])

In [ ]:
# Sorted by MaxDrawDown:
results_df.sort_values('MaxDrawDown',ascending=False).head()

In [ ]:
# Sorted by returns:
results_df.sort_values('returns',ascending=False).head()

In [ ]:
# Sorted by sharpe:
results_df.sort_values('sharpe',ascending=False).head()

In [ ]:
# Sorted by MaxDrawDownDuration:
results_df.sort_values('MaxDrawDownDuration',ascending=True).head()